In [1]:
import torch
from torchvision import models, transforms
import os
from pathlib import Path
from PIL import Image
import numpy as np
from tqdm import tqdm

In [2]:
model = models.resnet50(pretrained=True)
model.fc = torch.nn.Identity()
model.eval()

C:\Users\ABHINAV\AppData\Roaming\Python\Python310\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\ABHINAV\AppData\Roaming\Python\Python310\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\ABHINAV/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth


100%|█████████████████████████████████████████████████████████████████████████████| 97.8M/97.8M [00:28<00:00, 3.57MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [4]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [5]:
def extract_features(img_path, model):
    img = Image.open(img_path).convert("RGB")
    img_tensor = transform(img).unsqueeze(0)  # Add batch dimension
    with torch.no_grad():
        features = model(img_tensor)
    return features.squeeze().numpy()  # shape: (2048,)

In [6]:
def process_all_tracklets(crop_root, output_feat_path):
    tracklet_dirs = sorted(Path(crop_root).glob("*"))
    os.makedirs(output_feat_path, exist_ok=True)

    for tracklet_dir in tqdm(tracklet_dirs, desc="Extracting embeddings"):
        features = []
        for img_path in sorted(tracklet_dir.glob("*.jpg")):
            feat = extract_features(img_path, model)
            features.append(feat)

        # Aggregate (e.g., average) features across all frames
        tracklet_feat = np.mean(features, axis=0)

        # Save as .npy file (e.g., 001.npy)
        track_id = tracklet_dir.name
        np.save(Path(output_feat_path) / f"{track_id}.npy", tracklet_feat)


In [7]:
process_all_tracklets(
    crop_root=r"C:\Users\ABHINAV\Desktop\player-reid\Output", 
    output_feat_path=r"C:\Users\ABHINAV\Desktop\player-reid\tracklet_features"
)


Extracting embeddings:   0%|                                                                     | 0/2 [00:00<?, ?it/s]C:\Users\ABHINAV\AppData\Roaming\Python\Python310\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\ABHINAV\AppData\Roaming\Python\Python310\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
Extracting embeddings: 100%|█████████████████████████████████████████████████████████████| 2/2 [00:05<00:00,  2.58s/it]


In [9]:
!pip install torchreid


Defaulting to user installation because normal site-packages is not writeable


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
